In [1]:
import os
import pandas as pd
data_folder = os.path.join(os.path.expanduser("./ml-100k"), "ml-100k")
ratings_filename = os.path.join(data_folder, "u.data")

In [2]:
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings.head()

,UserID,MovieID,Rating,Datetime
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
all_ratings=all_ratings.drop(['Datetime'], axis=1)
all_ratings.head()

,UserID,MovieID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [7]:
ratings_final = all_ratings[all_ratings['UserID'].isin(range(100))]
ratings_final = ratings_final[ratings_final['MovieID'].isin(range(100))]
# ratings_final.to_csv('C:\\AAAAAA\\FINAL-RWR\\TRY1\\100users-100movies--.csv',index=False)

In [8]:
#all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'], unit='s')

In [9]:
all_ratings.head()

,UserID,MovieID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [10]:
all_ratings["Favorable"] = all_ratings["Rating"] > 3

In [11]:
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]
ratings.head()

,UserID,MovieID,Rating,Favorable
0,196,242,3,False
1,186,302,3,False
2,22,377,1,False
4,166,346,1,False
6,115,265,2,False


In [12]:
favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings.head()

,UserID,MovieID,Rating,Favorable
16,122,387,5,True
20,119,392,4,True
21,167,486,4,True
26,38,95,5,True
28,63,277,4,True


In [13]:
favorable_ratings = ratings[all_ratings["Favorable"]]
favorable_ratings.head()


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,UserID,MovieID,Rating,Favorable
16,122,387,5,True
20,119,392,4,True
21,167,486,4,True
26,38,95,5,True
28,63,277,4,True


In [14]:
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])

In [15]:
favorable_reviews_by_usersit=favorable_reviews_by_users.items()

In [16]:
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie.head()

,Favorable
MovieID,
1,66.0
2,5.0
3,4.0
4,21.0
5,6.0


In [17]:
num_favorable_by_movie.sort_values(by="Favorable", ascending=False).head()    #most trending

,Favorable
MovieID,
50,100.0
100,89.0
258,83.0
181,79.0
174,74.0


In [18]:
frequent_itemsets = {}

In [19]:
min_support = 40

In [20]:
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                            for movie_id, row in num_favorable_by_movie.iterrows()
                            if row["Favorable"] > min_support)                    #printing those of movies only that are freq
frequent_itemsets[1]                                                #frozen set-used as keys in counting dict(normal sets can't)

{frozenset({1}): 66.0,
 frozenset({7}): 67.0,
 frozenset({9}): 53.0,
 frozenset({12}): 45.0,
 frozenset({15}): 48.0,
 frozenset({22}): 42.0,
 frozenset({28}): 43.0,
 frozenset({50}): 100.0,
 frozenset({56}): 67.0,
 frozenset({64}): 58.0,
 frozenset({69}): 48.0,
 frozenset({79}): 58.0,
 frozenset({89}): 46.0,
 frozenset({96}): 46.0,
 frozenset({98}): 70.0,
 frozenset({100}): 89.0,
 frozenset({117}): 48.0,
 frozenset({121}): 50.0,
 frozenset({127}): 70.0,
 frozenset({135}): 47.0,
 frozenset({168}): 43.0,
 frozenset({172}): 59.0,
 frozenset({173}): 48.0,
 frozenset({174}): 74.0,
 frozenset({176}): 41.0,
 frozenset({181}): 79.0,
 frozenset({183}): 44.0,
 frozenset({191}): 46.0,
 frozenset({195}): 47.0,
 frozenset({204}): 44.0,
 frozenset({210}): 47.0,
 frozenset({216}): 46.0,
 frozenset({222}): 50.0,
 frozenset({237}): 50.0,
 frozenset({258}): 83.0,
 frozenset({269}): 47.0,
 frozenset({276}): 46.0,
 frozenset({286}): 59.0,
 frozenset({294}): 42.0,
 frozenset({300}): 46.0,
 frozenset({302})

In [21]:
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        a=list()
        for itemset in k_1_itemsets: #going through the prev discovered item sets and checking if it's a subset of 
            if itemset.issubset(reviews):
                other_reviewed_movie=reviews - itemset
                for i in other_reviewed_movie:
                    current_superset = itemset | frozenset((i,))
                    while current_superset not in a:
                        a.append(current_superset)
                        counts[current_superset] += 1         
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])
                   #test which of the candidate itemsets have support to be considered freq and returns only those

In [ ]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 40

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)

print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    #a=find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[1],min_support)
    #print(a)
                                                   
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()        #doesnt wait for entire code to complete
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

There are 45 movies with more than 40 favorable reviews


In [ ]:
#print(frequent_itemsets[1])

In [ ]:
print(frequent_itemsets[2])

In [21]:
print(frequent_itemsets[3])

{frozenset({56, 50, 174}): 44, frozenset({50, 174, 79}): 40, frozenset({50, 174, 98}): 42, frozenset({50, 172, 174}): 49, frozenset({50, 172, 181}): 45, frozenset({50, 181, 174}): 48, frozenset({50, 195, 174}): 42, frozenset({172, 181, 174}): 42}


In [22]:
print(frequent_itemsets[4])

{frozenset({50, 172, 181, 174}): 41}


In [23]:
frequent_itemsets.items()

dict_items([(1, {frozenset({1}): 66.0, frozenset({7}): 67.0, frozenset({9}): 53.0, frozenset({12}): 45.0, frozenset({15}): 48.0, frozenset({22}): 42.0, frozenset({28}): 43.0, frozenset({50}): 100.0, frozenset({56}): 67.0, frozenset({64}): 58.0, frozenset({69}): 48.0, frozenset({79}): 58.0, frozenset({89}): 46.0, frozenset({96}): 46.0, frozenset({98}): 70.0, frozenset({100}): 89.0, frozenset({117}): 48.0, frozenset({121}): 50.0, frozenset({127}): 70.0, frozenset({135}): 47.0, frozenset({168}): 43.0, frozenset({172}): 59.0, frozenset({173}): 48.0, frozenset({174}): 74.0, frozenset({176}): 41.0, frozenset({181}): 79.0, frozenset({183}): 44.0, frozenset({191}): 46.0, frozenset({195}): 47.0, frozenset({204}): 44.0, frozenset({210}): 47.0, frozenset({216}): 46.0, frozenset({222}): 50.0, frozenset({237}): 50.0, frozenset({258}): 83.0, frozenset({269}): 47.0, frozenset({276}): 46.0, frozenset({286}): 59.0, frozenset({294}): 42.0, frozenset({300}): 46.0, frozenset({302}): 48.0, frozenset({313})

In [37]:
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    print(itemset_length,itemset_counts,end="------")
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

1 {frozenset({1}): 66.0, frozenset({7}): 67.0, frozenset({9}): 53.0, frozenset({12}): 45.0, frozenset({15}): 48.0, frozenset({22}): 42.0, frozenset({28}): 43.0, frozenset({50}): 100.0, frozenset({56}): 67.0, frozenset({64}): 58.0, frozenset({69}): 48.0, frozenset({79}): 58.0, frozenset({89}): 46.0, frozenset({96}): 46.0, frozenset({98}): 70.0, frozenset({100}): 89.0, frozenset({117}): 48.0, frozenset({121}): 50.0, frozenset({127}): 70.0, frozenset({135}): 47.0, frozenset({168}): 43.0, frozenset({172}): 59.0, frozenset({173}): 48.0, frozenset({174}): 74.0, frozenset({176}): 41.0, frozenset({181}): 79.0, frozenset({183}): 44.0, frozenset({191}): 46.0, frozenset({195}): 47.0, frozenset({204}): 44.0, frozenset({210}): 47.0, frozenset({216}): 46.0, frozenset({222}): 50.0, frozenset({237}): 50.0, frozenset({258}): 83.0, frozenset({269}): 47.0, frozenset({276}): 46.0, frozenset({286}): 59.0, frozenset({294}): 42.0, frozenset({300}): 46.0, frozenset({302}): 48.0, frozenset({313}): 60.0, frozen

In [25]:
print(candidate_rules[100:140])

[(frozenset({98}), 174), (frozenset({127}), 100), (frozenset({100}), 127), (frozenset({174}), 100), (frozenset({100}), 174), (frozenset({174}), 172), (frozenset({172}), 174), (frozenset({181}), 172), (frozenset({172}), 181), (frozenset({174}), 181), (frozenset({181}), 174), (frozenset({174}), 195), (frozenset({195}), 174), (frozenset({50, 174}), 56), (frozenset({56, 174}), 50), (frozenset({56, 50}), 174), (frozenset({174, 79}), 50), (frozenset({50, 79}), 174), (frozenset({50, 174}), 79), (frozenset({98, 174}), 50), (frozenset({50, 98}), 174), (frozenset({50, 174}), 98), (frozenset({172, 174}), 50), (frozenset({50, 174}), 172), (frozenset({50, 172}), 174), (frozenset({172, 181}), 50), (frozenset({50, 181}), 172), (frozenset({50, 172}), 181), (frozenset({181, 174}), 50), (frozenset({50, 174}), 181), (frozenset({50, 181}), 174), (frozenset({195, 174}), 50), (frozenset({50, 174}), 195), (frozenset({50, 195}), 174), (frozenset({181, 174}), 172), (frozenset({172, 174}), 181), (frozenset({172

In [26]:
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():   
    for candidate_rule in candidate_rules:                  #where 1 and 2 are length 
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [27]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [28]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))
print(rule_confidence)

13
{(frozenset({172}), 50): 0.9152542372881356, (frozenset({195}), 174): 0.9361702127659575, (frozenset({56, 50}), 174): 0.9565217391304348, (frozenset({50, 79}), 174): 0.9523809523809523, (frozenset({172, 174}), 50): 0.9423076923076923, (frozenset({50, 172}), 174): 0.9074074074074074, (frozenset({172, 181}), 50): 0.9782608695652174, (frozenset({181, 174}), 50): 0.9230769230769231, (frozenset({195, 174}), 50): 0.9545454545454546, (frozenset({50, 195}), 174): 1.0, (frozenset({172, 181}), 174): 0.9130434782608695, (frozenset({172, 181, 174}), 50): 0.9761904761904762, (frozenset({50, 172, 181}), 174): 0.9111111111111111}


In [29]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [30]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({50, 195}) they will also recommend 174
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset({172, 181}) they will also recommend 50
 - Confidence: 0.978

Rule #3
Rule: If a person recommends frozenset({172, 181, 174}) they will also recommend 50
 - Confidence: 0.976

Rule #4
Rule: If a person recommends frozenset({56, 50}) they will also recommend 174
 - Confidence: 0.957

Rule #5
Rule: If a person recommends frozenset({195, 174}) they will also recommend 50
 - Confidence: 0.955



In [31]:
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join(data_folder, "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [32]:
movie_name_data.head()

,MovieID,Title,Release Date,Video Release,IMDB,<UNK>,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

In [34]:
get_movie_name(4)

'Get Shorty (1995)'

In [35]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Star Wars (1977), Terminator, The (1984) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 0.978

Rule #3
Rule: If a person recommends Empire Strikes Back, The (1980), Return of the Jedi (1983), Raiders of the Lost Ark (1981) they will also recommend Star Wars (1977)
 - Confidence: 0.976

Rule #4
Rule: If a person recommends Pulp Fiction (1994), Star Wars (1977) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 0.957

Rule #5
Rule: If a person recommends Terminator, The (1984), Raiders of the Lost Ark (1981) they will also recommend Star Wars (1977)
 - Confidence: 0.955



In [36]:
# Evaluation using test data
test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
test_favorable = test_dataset[test_dataset["Favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])

In [37]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [38]:
test_confidence = {candidate_rule:
                   (correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]))
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

13


In [39]:
for index in range(10):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Star Wars (1977), Terminator, The (1984) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.921

Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Train Confidence: 0.978
 - Test Confidence: 0.972

Rule #3
Rule: If a person recommends Empire Strikes Back, The (1980), Return of the Jedi (1983), Raiders of the Lost Ark (1981) they will also recommend Star Wars (1977)
 - Train Confidence: 0.976
 - Test Confidence: 0.973

Rule #4
Rule: If a person recommends Pulp Fiction (1994), Star Wars (1977) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 0.957
 - Test Confidence: 0.772

Rule #5
Rule: If a person recommends Terminator, The (1984), Raiders of the Lost Ark (1981) they will also recommend Star Wars (1977)
 - Train Confidence: 0.955
 - Test Confidence: 0.871

Rule #6
Rule: If a pers